In [ ]:
import numpy as np
import copy

import random

from py4j.java_gateway import JavaGateway
from collections import deque

import pad_discriminator_conformance

import statistics

In [ ]:
trace1 = ['A', 'B', 'D', 'E', 'I']
trace2 = ['A', 'C', 'D', 'G', 'H', 'F', 'I']
trace3 = ['A', 'C', 'G', 'D', 'H', 'F', 'I']
trace4 = ['A', 'C', 'H', 'D', 'F', 'I']
trace5 = ['A', 'C', 'D', 'H', 'F', 'I']


traces = [trace1, trace2, trace3, trace4, trace5]
frequencies = [1207, 145, 56, 23, 28]

GT_log = []
for f in range(0, len(frequencies)):
    print(traces[f], frequencies[f])
    for i in range(0, frequencies[f]):
        GT_log.append(traces[f])
        
random.shuffle(GT_log)

In [ ]:
class OutputGetter(deque):
    def appendleft(self, line):
        print(line)
        super().appendleft(line)
        
def remove_stuff(log): #removes short traces
    #new_log = [i for i in log if len(i) > 5]
    new_log = [i for i in log if len(i) > 1]
    return new_log
    
        
def remove_plus(dummylog):
    newlog = copy.deepcopy(dummylog)
    newlog = [[item.replace('+', '') for item in lst] for lst in newlog]
    return newlog
       
def fix_duplicates(dummylog):
    newlog = []
    for trace in dummylog:
        newtrace = trace[::2]
        newlog.append(newtrace)
    return newlog

def check_duplicates(dummylog): #not a good test but just to be sure
    if len(dummylog) > 1 and dummylog[0][0] == dummylog[0][1] and dummylog[1][0] == dummylog[1][1]:
        return True
    elif len(dummylog) == 1 and dummylog[0][0] == dummylog[0][1]:
        return True
    else:
        return False

In [ ]:
def get_model_log(modelfilename, size):
    gateway = JavaGateway.launch_gateway(classpath="./jars/*", 
                                         redirect_stdout=OutputGetter())
    petri_and_marking = gateway.jvm.org.processmining.plugins.kutoolbox.utils.ImportUtils.openPetrinet(
        gateway.jvm.java.io.File(modelfilename)
    )
    if not petri_and_marking[1].size():
        print("Creating initial marking")
        petri_and_marking[1] = gateway.jvm.org.processmining.plugins.kutoolbox.utils.PetrinetUtils. \
            getInitialMarking(petri_and_marking[0])

    petri_and_marking[1].size()
    settings = gateway.jvm.org.processmining.plugins.loggenerator.utils.GeneratorSettings()
    simulator = gateway.jvm.org.processmining.plugins.loggenerator.utils.GeneratorSettings.SimulationAlgorithm

    # See https://github.com/Macuyiko/processmining-prom/blob/master/loggenerator/
    #             org/processmining/plugins/loggenerator/utils/GeneratorSettings.java

    for t in petri_and_marking[0].getTransitions().iterator():
        isInvisible = t.getLabel() == "" or t.isInvisible()
        label = t.getLabel()
        isTextInvisible = label.startswith("inv_") or "$" in label
        mapped = label.replace("+complete", "").replace("\\ncomplete", "").replace("\\n", "") \
            if not isInvisible and not isTextInvisible else ""
        arr = gateway.new_array(gateway.jvm.int, 4)
        arr[0], arr[1], arr[2], arr[3] = 60, 0, 60, 0
        settings.getTransitionNames().put(t, mapped)
        settings.getTransitionWeights().put(t, 10)
        settings.getTransitionTimings().put(t, arr)
        #print(t.getId(), ':', label, '--->', mapped)

        settings.setNrTraces(size)
        settings.setRandomMinInGroup(1)
        settings.setRandomMaxInGroup(1)
        settings.setMustReachEnd(True)
        settings.setMustConsumeAll(False)
        settings.setMaxTimesMarkingSeen(7)

        settings.setSimulationMethod(simulator.Random) # Random, Complete, or Distinct

        # settings.setRestartAfter(1000)
        # settings.setSkipChance(0.85)
        # settings.setAlsoConsiderPartial(False)
    xlog = gateway.jvm.org.processmining.plugins.loggenerator.PetriNetLogGenerator.generate(
        petri_and_marking[0], 
        petri_and_marking[1], 
        settings, 
        #None # or for output: gateway.jvm.org.processmining.plugins.kutoolbox.eventlisteners.PluginEventListenerCLI()
        gateway.jvm.org.processmining.plugins.kutoolbox.eventlisteners.PluginEventListenerCLI()
    )

    temp_log = []
    for trace in xlog:
        #print([gateway.jvm.org.deckfour.xes.extension.std.XConceptExtension.instance().extractName(e) for e in trace])
        temp_log.append([gateway.jvm.org.deckfour.xes.extension.std.XConceptExtension.instance().extractName(e) for e in trace])

    model_log = remove_stuff(temp_log)


    if check_duplicates(model_log) == True:
        print('Fixing duplicates')
        model_log = fix_duplicates(model_log)
    return(model_log)

In [ ]:
def test2(GT_log, filename, times, modellogsize): #random model log generation
    fitness_results1 = []
    precision_results1 = []
    fitness_results2 = []
    precision_results2 = []
    
    for i in range(0, times):
        print(i, " OUT OF ", times)
        model_log = get_model_log(filename, modellogsize)
        fitness1, fitness2 = pad_discriminator_conformance.get_distance(GT_log, model_log, "random")
        precision1, precision2 = pad_discriminator_conformance.get_distance(model_log, GT_log, "random")
        fitness_results1.append(fitness1)
        fitness_results2.append(fitness2)
        precision_results1.append(precision1)
        precision_results2.append(precision2)
    print("FITNESS PROB VALUE:",statistics.median(fitness_results1), statistics.mean(fitness_results1), np.std(fitness_results1))
    print("FITNESS COUNT VALUE:",statistics.median(fitness_results2), statistics.mean(fitness_results2), np.std(fitness_results2))
    print("PREC PROB VALUE:", statistics.median(precision_results1), statistics.mean(precision_results1), np.std(precision_results1))
    print("PREC COUNT VALUE:", statistics.median(precision_results2), statistics.mean(precision_results2), np.std(precision_results2))

In [ ]:
test(GT_log, "Model1.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model2.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model3.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model4.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model5.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model6.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model7.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model8.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model9.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model10.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model11.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model_alpha_miner.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model_Inductiive_noise0.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model_Inductiive_noise0-5.pnml", 10, 1500)

In [ ]:
test(GT_log, "Model_Inductiive_noise1.pnml", 10, 1500)